In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import f , t, norm
import scipy.stats
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go



In [ ]:
df = pd.read_csv('SalaryMulti.csv')
df.head()

,Total Experience,Team Lead Experience,Project Manager Experience,Certifications,Salary
0,7,2,4,1,77318.070547
1,4,0,2,3,64951.950980
2,13,4,8,3,106058.185204
3,11,3,2,1,89649.944851
4,8,1,6,3,82206.019687


In [ ]:
df.shape


(1000, 5)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Total Experience            1000 non-null   int64  
 1   Team Lead Experience        1000 non-null   int64  
 2   Project Manager Experience  1000 non-null   int64  
 3   Certifications              1000 non-null   int64  
 4   Salary                      1000 non-null   float64
dtypes: float64(1), int64(4)
memory usage: 39.2 KB


In [ ]:
X= df[['Total Experience','Team Lead Experience','Project Manager Experience','Certifications']].values
y = df['Salary'].values

In [ ]:
scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_scaled = scaler_x.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten() # flatten to 1D

In [ ]:
class MultipleLinearRegression():
    def __init__(self):
        self.B = None
        self.MSE = None



    def fit(self, X, y):
      # Ensure X and y are arrays
        self.X = np.array(X)
        self.y = np.array(y)

        #calculate df
        self.n= self.X.shape[0]   # no. of rows
        self.k = self.X.shape[1]  # no. of features
        self.p= self.k +1         # no. of parameteres

        #  Calculate the mean of y
        y_bar = np.mean(y)

        # Add intercept term
        X= np.hstack([np.ones((X.shape[0], 1)), X])
        self.X_design = X


        y_t =y.T           #calculate y-transpose
        y_t_y = y_t @  y    #calculate y-transpose* y
        X_t = X.T         #calculate X-transpose
        X_t_X = X_t @ X   #calculate X-transpose* X
        self.X_t_X_inv = np.linalg.inv(X_t_X) #calculate inverse
        X_t_y = X_t @ y

        # calculate B
        B = self.X_t_X_inv @ X_t @ y
        self.B = B


       # calculate B-transpose
        B_t = B.T



       # calculate Sum Squares
        self.SST = y_t_y -len(y) * (y_bar**2)
        self.SSE = y_t_y - B_t @ X_t @ y
        self.SSR = self.SST - self.SSE



        # Calculate MSE
        df_Error = self.n - self.p
        self.MSE = self.SSE / df_Error

        # Example: after model.fit(X, y)
        coefs = model.B.flatten()
        equation = f"ŷ = {coefs[0]:.4f}"  # Intercept

        for i, coef in enumerate(coefs[1:], start=1):
            equation += f" + ({coef:.4f})*x{i}"

        print("Final Regression Equation:")
        print(equation)






    def predict(self, X):

        #  Raise a ValueError if the model has not been fitted yet
        if self.B is None :
              raise ValueError("The model has not been fitted yet.")


        if np.linalg.det(X.T @ X) == 0 :
              raise ValueError("Can't Calculate inverse.")

        

        X = np.array(X)

        # Add intercept (bias) term
        X = np.hstack([np.ones((X.shape[0], 1)), X])

        if X.shape[1] != self.B.shape[0]:
             raise ValueError(f"Mismatch in feature dimensions: X has {X.shape[1]} columns, but model expects {self.B.shape[0]}.")


        #Corrected the matrix multiplication order
        return X @ self.B

    def plot(self, feature_indices=[0, 1]):

        if self.B is None:
            raise ValueError("The model has not been fitted yet.")
        if self.X.shape[1] < 2:
            raise ValueError("Need at least two features for 3D plot.")

        # Extract the two scaled features
        x1 = self.X[:, feature_indices[0]]
        x2 = self.X[:, feature_indices[1]]

        # Create a meshgrid using the feature ranges
        x1_grid, x2_grid = np.meshgrid(
            np.linspace(x1.min(), x1.max(), 20),
            np.linspace(x2.min(), x2.max(), 20)
        )

        # Create X_grid with all features
        num_features = self.X.shape[1]
        X_grid = np.zeros((x1_grid.size, num_features))
        X_grid[:, feature_indices[0]] = x1_grid.ravel()
        X_grid[:, feature_indices[1]] = x2_grid.ravel()

        # Fill other features with their mean
        for i in range(num_features):
            if i not in feature_indices:
                X_grid[:, i] = self.X[:, i].mean()

        # Predict
        y_pred_grid = self.predict(X_grid).reshape(x1_grid.shape)

        # Flatten actual values for scatter plot
        y_actual = self.y.flatten()

        # Interactive Plotly plot
        fig = go.Figure()

        # Actual data points
        fig.add_trace(go.Scatter3d(
            x=x1, y=x2, z=y_actual,
            mode='markers',
            marker=dict(size=6, color='blue'),
            name='Actual'
        ))

        # Regression surface
        fig.add_trace(go.Surface(
            x=x1_grid, y=x2_grid, z=y_pred_grid,
            colorscale='Reds',
            opacity=0.6,
            name='Regression Surface'
        ))

        # Layout
        fig.update_layout(
            title='Interactive 3D Regression Plot',
            scene=dict(
                xaxis_title=f'Feature {feature_indices[0]}',
                yaxis_title=f'Feature {feature_indices[1]}',
                zaxis_title='Target'
            ),
            width=800,
            height=600
        )

        fig.show()




    def ANOVA_table(self):

        #  Raise a ValueError if the model has not been fitted yet
        if self.B is None :
              raise ValueError("The model has not been fitted yet.")

        #calculate df
        df_Regression = self.k
        df_Error = self.n - self.p
        df_Total = self.n - 1

       #calculate Mean Squares
        MSE = self.SSE / df_Error
        MSR = self.SSR / df_Regression

        #calculate F-statistic
        self.F_stat = MSR / MSE

        #calculate p-value
        self.p_value = 1 - scipy.stats.f.cdf(self.F_stat, df_Regression, df_Error)

          # Assemble the ANOVA table
        ANOVA_Data = {
           'Source': ['Regression', 'Error', 'Total'],
           'Sum of Squares': [self.SSR, self.SSE, self.SST],
           'Degrees of Freedom': [df_Regression, df_Error, df_Total],
           'Mean Square': [MSR, MSE, ""],
           'F-Statistic': [self.F_stat, "", ""],
           'p-value': [self.p_value, "", ""]
       }
        ANOVA_table = pd.DataFrame(ANOVA_Data)
        return ANOVA_table



    def feature_ANOVA_table(self):
        # Check if the model is fitted
        if self.B is None:
            raise ValueError("The model has not been fitted yet.")

        # Get the design matrix (including intercept)
        X = self.X_design
        n, p = X.shape
        k = p - 1  # Number of features (assuming first column is intercept)


         # Calculate MSE
        df_Error = self.n - self.p
        MSE = self.SSE / df_Error

        ANOVA_data = []
        for i in range(1, p):  # Loop through features (skip intercept)
            beta_i = self.B[i]
            xtx_inv_ii = self.X_t_X_inv[i, i]
            SS_i = (beta_i ** 2) / xtx_inv_ii
            df_i = 1
            MS_i = SS_i / df_i
            F_i = MS_i / MSE
            p_value_i = 1 - scipy.stats.f.cdf(F_i, df_i, df_Error)
            ANOVA_data.append({
                'Feature': f'Feature {i}',
                'Sum of Squares': SS_i,
                'Degrees of Freedom': df_i,
                'Mean Square': MS_i,
                'F-Statistic': F_i,
                'p-value': p_value_i
            })

        # Create DataFrame
        ANOVA_table = pd.DataFrame(ANOVA_data)
        return ANOVA_table





    def hypothesis_test(self , alpha = .05):

        """The null hypothesis (H_0) assumes that there is no relationship bet ween x and y,

          while the alternative hypothesis (H_a) suggests that there is a relationship."""


        # Calculate critical value and p-value
        F_critical = f.ppf(1 - alpha, self.k, self.n -self.p)
        p_value = f.sf(self.F_stat, self.k, self.n -self.p)

        # Determine conclusion
        if self.F_stat > F_critical:
            conclusion = ("Reject H₀: Significant evidence that at least one\n"
                          "predictor has a relationship with the response.")
        else:
            conclusion = ("Fail to reject H₀: No significant evidence that\n"
                         "any predictors are related to the response.")

        # Print formatted results
        print("\n         Hypothesis Testing Results (Overall Model)")
        print("         -----------------------------------------")
        print(f"{'Null Hypothesis (H₀):':<30} All β_j = 0 (j = 1,...,k)")
        print(f"{'Alternative Hypothesis (Hₐ):':<30} At least one β_j ≠ 0")
        print(f"{'Significance Level (α):':<30} {alpha}")
        print(f"{'F-statistic (F₀):':<30} {self.F_stat:.4f}")
        print(f"{'Critical Value (F_c):':<30} {F_critical:.4f}")
        print(f"{'p-value:':<30} {p_value:.4f}")
        print("\nConclusion:", conclusion)

    def interval_estimation(self, alpha=0.05, sigma=None):
            if self.B is None:
                raise ValueError("Model has not been fitted yet.")

            df = self.n - self.p  # degrees of freedom

            # Get critical value
            if sigma is None:
                t = scipy.stats.t.ppf(1 - alpha / 2, df)
            else:
                z = scipy.stats.norm.ppf(1 - alpha / 2)

            # Compute intervals
            intervals = []
            for i in range(self.p):
                var_Bi = self.X_t_X_inv[i, i]
                if sigma is None:
                    margin = t * np.sqrt(self.MSE * var_Bi)
                else:
                    margin = z * sigma * np.sqrt(var_Bi)

                # Access the coefficient directly since it's a scalar
                lower = self.B[i] - margin
                upper = self.B[i] + margin
                intervals.append((lower, upper))

            # Print intervals with labels
            for i, (low, high) in enumerate(intervals):
                print(f"Confidence interval for B{i}: ({low:.4f}, {high:.4f})")

            return intervals










In [ ]:
model = MultipleLinearRegression()
model.fit(X_scaled, y_scaled)

Final Regression Equation:
ŷ = 0.0000 + (0.6735)*x1 + (0.3267)*x2 + (0.1165)*x3 + (0.0304)*x4


In [ ]:
y_pred_scaled = model.predict(X_scaled)  # Predict using scaled features
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()  # Inverse transform to original scale


In [ ]:
df['predicted_Salary'] = y_pred
df.head()

,Total Experience,Team Lead Experience,Project Manager Experience,Certifications,Salary,predicted_Salary
0,7,2,4,1,77318.070547,79730.622330
1,4,0,2,3,64951.950980,65739.825284
2,13,4,8,3,106058.185204,106227.840456
3,11,3,2,1,89649.944851,91619.627109
4,8,1,6,3,82206.019687,83567.126837


In [38]:
model.plot()
print(model.ANOVA_table())
print(model.feature_ANOVA_table())
model.hypothesis_test()
model.interval_estimation()

       Source  Sum of Squares  Degrees of Freedom Mean Square F-Statistic  \
0  Regression      928.155971                   4  232.038993   3213.6115   
1       Error       71.844029                 995    0.072205               
2       Total     1000.000000                 999                           

  p-value  
0     0.0  
1          
2          
     Feature  Sum of Squares  Degrees of Freedom  Mean Square  F-Statistic  \
0  Feature 1      124.956257                   1   124.956257  1730.574929   
1  Feature 2       37.700804                   1    37.700804   522.135243   
2  Feature 3        6.445688                   1     6.445688    89.269209   
3  Feature 4        0.923242                   1     0.923242    12.786387   

        p-value  
0  1.110223e-16  
1  1.110223e-16  
2  1.110223e-16  
3  3.659389e-04  

         Hypothesis Testing Results (Overall Model)
         -----------------------------------------
Null Hypothesis (H₀):          All β_j = 0 (j = 1,...,k)
A

[(np.float64(-0.016674794754496958), np.float64(0.016674794754497235)),
 (np.float64(0.6417634274527664), np.float64(0.7053070257610627)),
 (np.float64(0.2986093411264669), np.float64(0.3547160409750714)),
 (np.float64(0.0922839547214725), np.float64(0.1406665990990023)),
 (np.float64(0.013718279098341026), np.float64(0.04708776599084659))]